# VAE Training

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICE"]="0,1"

In [8]:
import torch as t
import torch.nn.functional as F
from tqdm import tqdm
from models.VariationalAutoencoder import VariationalAutoencoder
from torchvision import datasets, transforms

bs = 128
train_ds = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
train_dl = t.utils.data.DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, drop_last=True)

device = t.device('cuda') if t.cuda.is_available() else 'cpu'
model = VariationalAutoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
                    enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 0, 1], enc_strides=[1, 2, 2, 1],
                    dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
                    dec_pads=[1, 0, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
model.cuda(device)
model.train()

VariationalAutoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(nega

In [9]:
train_ds

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [10]:
60000/bs/33

14.204545454545455

In [24]:
def vae_kl_loss(mu, log_var):
    return -.5 * t.sum(1 + log_var - mu ** 2 - log_var.exp())

def vae_loss(y_pred, mu, log_var, y_true):
    r_loss = F.binary_cross_entropy(y_pred, y_true, reduction='sum')
    r_loss_factor = 1
    kl_loss = vae_kl_loss(mu, log_var)
    return r_loss_factor*r_loss + kl_loss

In [25]:
lr = .0005
for epoch in tqdm(range(20)):
    optimizer = t.optim.Adam(model.parameters(), lr=lr / (epoch * 2 + 1), betas=(.9, .99), weight_decay=1e-2)
    for i, (data, _) in enumerate(train_dl):
        data = data.to(device)
        optimizer.zero_grad()
        pred, mu, log_var = model(data)
        loss = vae_loss(pred, mu, log_var, data)
        loss.backward()
        optimizer.step()
        if i % 33 == 0:
            print(loss)

print(loss)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

tensor(27425.9941, device='cuda:0', grad_fn=<AddBackward0>)
tensor(27194.8574, device='cuda:0', grad_fn=<AddBackward0>)
tensor(26925.9551, device='cuda:0', grad_fn=<AddBackward0>)
tensor(27584.6895, device='cuda:0', grad_fn=<AddBackward0>)
tensor(27446.9551, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25211.4531, device='cuda:0', grad_fn=<AddBackward0>)
tensor(26710.0527, device='cuda:0', grad_fn=<AddBackward0>)
tensor(27481.5137, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24401.8633, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24927.9707, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25145.7012, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23197.8594, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23601.8770, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22855.0430, device='cuda:0', grad_fn=<AddBackward0>)


  5%|██▏                                         | 1/20 [00:16<05:19, 16.79s/it]

tensor(23399.7520, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21946.8398, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22132.9160, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22751.9805, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22531.1582, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22632.1699, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22708.0586, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22096.5254, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21962.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21613.1719, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21507.3164, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21882.6094, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21843.3281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22493.6484, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21218.6641, device='cuda:0', grad_fn=<AddBackward0>)


 10%|████▍                                       | 2/20 [00:33<05:03, 16.83s/it]

...

tensor(20408.6836, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22111.8750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20995.6523, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21709.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20714.3281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20860.6914, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21295.3105, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21339.0039, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20850.4941, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20414.3770, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21192.6621, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22407.6133, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21123.0332, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20852.9082, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21873.9062, device='cuda:0', grad_fn=<AddBackward0>)


 95%|████████████████████████████████████████▊  | 19/20 [05:23<00:17, 17.01s/it]

tensor(20584.6582, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21277.3301, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20750.3711, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20366.0449, device='cuda:0', grad_fn=<AddBackward0>)
tensor(19736.5410, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21710.2266, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20330.7715, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21635.8164, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21919.7891, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20429.6816, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20928.4395, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21645.4668, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21156.7246, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21668.0547, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20927.2285, device='cuda:0', grad_fn=<AddBackward0>)


100%|███████████████████████████████████████████| 20/20 [05:37<00:00, 16.88s/it]

tensor(20646.6816, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21408.9414, device='cuda:0', grad_fn=<AddBackward0>)


In [13]:
mu.shape

torch.Size([128, 2])

In [14]:
log_var.shape

torch.Size([128, 2])

In [15]:
mu[0:10]

tensor([[ 0.2095, -2.3853],
        [ 1.2137, -2.5154],
        [ 0.4480, -0.9542],
        [ 0.1077, -4.3537],
        [-0.9446, -2.6549],
        [ 1.4333, -0.6426],
        [ 3.2849, -1.8471],
        [-0.0786, -1.9685],
        [-0.1759, -0.9804],
        [-0.1184, -1.4328]], device='cuda:0', grad_fn=<SliceBackward0>)

In [16]:
log_var[0:10]

tensor([[-4.1537, -3.4211],
        [-3.1524, -3.2208],
        [-3.1086, -2.8434],
        [-4.3717, -2.1544],
        [-3.8358, -5.3702],
        [-2.8891,  0.5710],
        [-3.0989, -3.4410],
        [-3.6518, -3.1645],
        [-4.0692, -1.9565],
        [-2.7796, -4.3813]], device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
std = t.exp(log_var[0:10]/2)
std

In [ ]:
loss

In [ ]:
print('20_epoch_loss=21229')

In [ ]:
t.save(model.state_dict(), 'models/state_dicts/03_03.pth')

In [ ]:
m = t.Tensor([0, 0])

In [ ]:
log_va = t.Tensor([1, 1])

In [ ]:
-.5 * t.sum(1 + log_va - m ** 2 - log_va.exp())

In [ ]:
log_va - m ** 2 - log_va.exp()

In [23]:
t.sum(log_va - m ** 2 - log_va.exp())

tensor(-3.4366)